In [4]:
%matplotlib inline
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)


In [103]:
# Nos données 
iata_state = pd.DataFrame({'IATA': {0: 'PHL', 1: 'DFW', 2: 'IAH', 3: 'ATL', 4: 'BOS', 5: 'LAX', 6: 'SFO', 7: 'EWR', 8: 'JFK', 9: 'LGA', 10: 'DTW', 11: 'DEN', 12: 'CLT', 13: 'MSP', 14: 'SEA', 15: 'ORD', 16: 'MCO', 17: 'MIA', 18: 'LAS', 19: 'PHX'}, 'STATE_FULL': {0: 'Pennsylvania', 1: 'Texas', 2: 'Texas', 3: 'Georgia', 4: 'Massachusetts', 5: 'California', 6: 'California', 7: 'New Jersey', 8: 'New York', 9: 'New York', 10: 'Michigan', 11: 'Colorado', 12: 'North Carolina', 13: 'Minnesota', 14: 'Washington', 15: 'Illinois', 16: 'Florida', 17: 'Florida', 18: 'Nevada', 19: 'Arizona'}})
iata_state = iata_state.rename(columns={'STATE_FULL':'State'})
iata_state

,IATA,State
0,PHL,Pennsylvania
1,DFW,Texas
2,IAH,Texas
3,ATL,Georgia
4,BOS,Massachusetts
5,LAX,California
6,SFO,California
7,EWR,New Jersey
8,JFK,New York
9,LGA,New York


## Données temportelles : Unemployement 

In [80]:

# https://www.kaggle.com/jayrav13/unemployment-by-county-us/data#output.csv
# Taux de chomage tout les mois de 1990 à 2016 tout les mois pour chaque County (compté) des States
unemployment = pd.read_csv(
    os.path.join('data', 'unemployment.csv')
).rename(columns={'Rate':'unemployment'})
unemployment.head()
def monthToNum(shortMonth):
    return{
            'January' : 1,
            'February' : 2,
            'March' : 3,
            'April' : 4,
            'May' : 5,
            'June' : 6,
            'July' : 7,
            'August' : 8,
            'September' : 9, 
            'October' : 10,
            'November' : 11,
            'December' : 12
    }[shortMonth]
unemployment['Month'] = unemployment['Month'].apply(lambda month : monthToNum(month))
unemployment

,Year,Month,State,County,unemployment
0,2015,2,Mississippi,Newton County,6.1
1,2015,2,Mississippi,Panola County,9.4
2,2015,2,Mississippi,Monroe County,7.9
3,2015,2,Mississippi,Hinds County,6.1
4,2015,2,Mississippi,Kemper County,10.6
...,...,...,...,...,...
885543,2009,11,Maine,Somerset County,10.5
885544,2009,11,Maine,Oxford County,10.5
885545,2009,11,Maine,Knox County,7.5
885546,2009,11,Maine,Piscataquis County,11.3


In [81]:
#unemployment = unemployment[unemployment['State'].isin(STATE_FULL)]
#sate_un = unemployment[unemployment['State'].isin(STATE_FULL)].State.unique()
#print("Il nous manque les états de : ",np.setdiff1d(STATE_FULL, sate_un))

# La floride est voisine de la Géorigie et de l'Alabama, la Georgie etant aussi manquante
# nous allons remplacer Floride par Alabama qui ont des taux de chomage comparable depuis 1990. 
#https://www.google.com/publicdata/explore?ds=z1ebjpgk2654c1_&met_y=unemployment_rate&fdim_y=seasonality:S&idim=state:ST1200000000000:ST0600000000000&hl=en&dl=en#!ctype=l&strail=false&bcs=d&nselm=h&met_y=unemployment_rate&fdim_y=seasonality:S&scale_y=lin&ind_y=false&rdim=state&idim=state:ST1200000000000:ST0100000000000&ifdim=state:country:US&hl=en_US&dl=en&ind=false
unemployment_florida = unemployment[unemployment['State'] == 'Alabama']
unemployment_florida['State'] = 'Florida'

# De meme pour la georgie qui est voisine avec : North Carolina, Alabama, South Carolina, Tennessee
# https://www.google.com/publicdata/explore?ds=z1ebjpgk2654c1_&met_y=unemployment_rate&fdim_y=seasonality:S&idim=state:ST1200000000000:ST0600000000000&hl=en&dl=en#!ctype=l&strail=false&bcs=d&nselm=h&met_y=unemployment_rate&fdim_y=seasonality:S&scale_y=lin&ind_y=false&rdim=state&idim=state:ST0100000000000:ST1300000000000:ST4500000000000:ST4700000000000:ST3700000000000&ifdim=state:country:US&hl=en_US&dl=en&ind=false
unemployment_georgia = unemployment[unemployment['State'].isin(['North Carolina','Alabama','South Carolina', 'Tennessee',])]
unemployment_georgia['State'] = 'Georgia'
unemployment_georgia
unemployment = pd.concat([unemployment,unemployment_georgia, unemployment_florida])

# on ne garde que les état qui nous interessent 
#unemployment = unemployment[unemployment['State'].isin(STATE_FULL)]
unemployment.head()

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year,Month,State,County,unemployment
0,2015,2,Mississippi,Newton County,6.1
1,2015,2,Mississippi,Panola County,9.4
2,2015,2,Mississippi,Monroe County,7.9
3,2015,2,Mississippi,Hinds County,6.1
4,2015,2,Mississippi,Kemper County,10.6


In [317]:
# selection des states
print(len(iata_state.State.unique()))
unemployment = unemployment[unemployment['State'].isin(iata_state.State.unique())]


16


In [318]:
state_unemployment = unemployment.groupby(['Year','State','Month']).mean().reset_index()
print(state_unemployment.shape)
print(state_unemployment[(state_unemployment['Year'] == 1990) & (state_unemployment['Month'] == 1)].State.nunique())
state_unemployment[(state_unemployment['Year'] == 1990) & (state_unemployment['Month'] == 1)].State.unique()
state_unemployment.head()

(5022, 4)
16


,Year,State,Month,unemployment
0,1990,Arizona,1,8.540000
1,1990,Arizona,2,8.333333
2,1990,Arizona,3,7.893333
3,1990,Arizona,4,7.620000
4,1990,Arizona,5,7.480000


In [319]:
state_unemployment[state_unemployment.Year.isin(['2012','2013']) & (state_unemployment.State == 'California')]
month = pd.DataFrame(list(range(1,13)), columns=["Month"])
month['Year'] = 2012
month['State'] = 'California'

california_unemployment_2012 = state_unemployment.merge(month, 
                   how='right',
                   on=["Year",'Month', 'State'],
                   )

month = pd.DataFrame(list(range(1,13)), columns=["Month"])
month['Year'] = 2013
month['State'] = 'California'
california_unemployment_2013 = state_unemployment.merge(month, 
                   how='right',
                   on=["Year",'Month', 'State'],
                   )
california_unemployment_2013
california_unemployment_na = pd.concat([california_unemployment_2012,california_unemployment_2013])
#california_unemployment_na['unemployment'] = 
california_unemployment_na.unemployment = california_unemployment_na.apply(lambda row : 
    state_unemployment[(state_unemployment.Year == row.Year) & (state_unemployment.Month == row.Month) & (state_unemployment.State.isin(['Arizona','Nevada']))].mean().unemployment,
                                                              axis = 1)
california_unemployment_na.head()

,Year,State,Month,unemployment
0,2012,California,5,10.606275
1,2012,California,6,10.996863
2,2012,California,10,10.142941
3,2012,California,1,11.696471
4,2012,California,2,11.254706


In [320]:
month = pd.DataFrame(list(range(1,13)), columns=["Month"])
month['Year'] = 2012
month['State'] = 'Texas'

texas_unemployment_na = state_unemployment.merge(month, 
                   how='right',
                   on=["Year",'Month', 'State'],
                   )

# ici on remplce les na par le min, en effet sur cette periode le chomage ne cesse de baisser
texas_unemployment_na.fillna((texas_unemployment_na['unemployment'].min()), inplace=True)
texas_unemployment_na.head()


,Year,State,Month,unemployment
0,2012,Texas,1,7.016929
1,2012,Texas,2,6.845276
2,2012,Texas,5,6.357480
3,2012,Texas,6,7.026772
4,2012,Texas,7,7.006693


In [321]:
# on supprime les données de notre dataset
state_unemployment = state_unemployment.drop(state_unemployment[((state_unemployment.Year == 2012) & (state_unemployment.State == 'Texas'))].index)
state_unemployment = state_unemployment.drop(state_unemployment[(state_unemployment['Year'].isin([2012,2013]) & (state_unemployment.State == 'California'))].index)

# on merge nos nouvelle données
state_unemployment = pd.concat([state_unemployment, texas_unemployment_na, california_unemployment_na])
state_unemployment.head()

In [322]:
iata_unemployment = iata_state.merge(state_unemployment)
iata_unemployment = iata_unemployment.drop(columns="State")
#verif
print("IATA : ",iata_unemployment.IATA.nunique())
iata_unemployment.head()

IATA :  20


,IATA,Year,Month,unemployment
0,PHL,1990,1,7.398507
1,PHL,1990,2,7.438806
2,PHL,1990,3,6.798507
3,PHL,1990,4,6.179104
4,PHL,1990,5,5.997015


In [323]:
external_data = pd.read_csv('submissions/submit/external_data.csv', 
                            index_col=0)
external_data["year"] = pd.to_datetime(external_data["Date"]).dt.year
external_data["month"] = pd.to_datetime(external_data["Date"]).dt.month
print(external_data.shape)
external_data = external_data.merge(iata_unemployment, 
                   how='left',
                   left_on=["year",'month',"AirPort"],
                   right_on= ["Year",'Month',"IATA"]
                   ).drop(columns=["Year",'Month',"IATA"])
print(external_data.shape)
external_data.head()

(11040, 29)
(11040, 30)


,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Holiday,year,month,TOTAL_DEPARTURE,TOTAL_DESTINATION,unemployment
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,0,2011,9,3582088,3580851,10.995779
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,0,2011,9,3582088,3580851,10.995779
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,0,2011,9,3582088,3580851,10.995779
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,0,2011,9,3582088,3580851,10.995779
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,2011,9,3582088,3580851,10.995779


In [324]:
print('unemployment na : ',external_data.unemployment.isnull().sum())
na_unemployment = external_data[external_data.unemployment.isnull()][['Date','AirPort','unemployment', 'year']]
print(na_unemployment.AirPort.unique())
na_unemployment

unemployment na :  0
[]


,Date,AirPort,unemployment,year


In [328]:
#external_data[(external_data['AirPort'] == "ATL") & (external_data['year'] == "2011")& (external_data['month'] == "9")]
print("external_data 2011 09 ATL Rate : ",
    external_data[(external_data['AirPort'] == "ATL") & (external_data['year'] == 2011)& (external_data['month'] == 9)].unemployment.unique()
     )

print("iata_unemployment 2011 09 ATL Rate : ",
    iata_unemployment[(iata_unemployment['IATA'] == "ATL") & (iata_unemployment['Year'] == 2011)& (iata_unemployment['Month'] == 9)].unemployment.unique()
     )
external_data

external_data 2011 09 ATL Rate :  [10.99577922]
iata_unemployment 2011 09 ATL Rate :  [10.99577922]


,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Holiday,year,month,TOTAL_DEPARTURE,TOTAL_DESTINATION,unemployment
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,0,2011,9,3582088,3580851,10.995779
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,0,2011,9,3582088,3580851,10.995779
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,0,2011,9,3582088,3580851,10.995779
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,0,2011,9,3582088,3580851,10.995779
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,2011,9,3582088,3580851,10.995779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,0,2013,3,1164319,1141295,8.604839
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,0,2013,3,1164319,1141295,8.604839
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,0,2013,3,1164319,1141295,8.604839
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,0,2013,3,1164319,1141295,8.604839


In [330]:
#external_data.to_csv('submissions/submit/external_data_unemployment.csv')
external_data.to_csv('submissions/submit/external_data.csv')
